In [8]:
from bluesky import RunEngine
from bluesky.callbacks import LiveTable, LivePlot, CallbackBase
import classes

In [9]:
RE = RunEngine()

ps = classes.PowerSupply("G3:PS:", name="ps")
ps.turn_on()

# attiva la tabella
fields = ["ps_wrCurRamp","ps_rdCur","ps_rdVolt","ps_rdHeatSinkTmp","ps_rdHeatShuntTmp","ps_done"]
tok = RE.subscribe(LiveTable(fields))

# esempio: 0→5 A, step 1 A
uid, = RE(classes.ramp_scan(ps, 0, 5, 1, pre_wait_s=2, hold_s=1, tol=0.02, base_timeout=5))
print("UID:", uid)

# disattiva la tabella
tok = RE.unsubscribe(LiveTable(fields)) 

rasp = classes.Gaussmeter("G3:GM:", name = "rasp")
rasp.wait_for_connection()

# Switch the GM on
rasp.switch(1)



+-----------+------------+--------------+------------+------------+------------------+-------------------+------------+
|   seq_num |       time | ps_wrCurRamp |   ps_rdCur |  ps_rdVolt | ps_rdHeatSinkTmp | ps_rdHeatShuntTmp |    ps_done |
+-----------+------------+--------------+------------+------------+------------------+-------------------+------------+
|         1 | 10:03:40.9 |            0 |      0.000 |      0.004 |           32.850 |            32.210 |          0 |
|         2 | 10:03:45.1 |            1 |      1.000 |      2.745 |           32.850 |            32.140 |          0 |
|         3 | 10:03:49.0 |            2 |      2.000 |      5.484 |           32.850 |            32.140 |          0 |
|         4 | 10:03:53.1 |            3 |      3.000 |      8.224 |           32.850 |            32.140 |          0 |
|         5 | 10:03:57.1 |            4 |      4.000 |     10.966 |           32.850 |            32.280 |          0 |
|         6 | 10:04:01.1 |            

In [7]:

import time
from datetime import datetime 
import csv
import os
#import path



class CSVWriter(CallbackBase):
    def __init__(self, fields, fname_format, fpath):
        # calculated in the beginning
        self._path = path.Path(fpath)
        os.makedirs(self._path, exist_ok=True)
        self._fname_fomat = fname_format
        self._fields = fields
        self._writer = None
        self._fout = None

    def close(self):
        if self._fout is not None:
            self._fout.close()
        self._fout = None
        self._writer = None

    def start(self, doc):
        self.close()

        fname = self._path / self._fname_fomat.format(**doc)

        self._fout = open(fname, 'xt')
        self._writer = csv.writer(self._fout)

    def descriptor(self, doc):
        if self._writer is not None:
            self._writer.writerow(self._fields)

    def event(self, doc):
        data = doc['data']
        if self._writer is not None:
            self._writer.writerow(data[k] for k in self._fields)

    def stop(self, doc):
        self.close()

def create_cbs():
    return LiveTable(fields)

timestamp = datetime.now().strftime('%Y%m%d%H%M')
print(timestamp)

NameError: name 'CallbackBase' is not defined